In [ ]:
import tensorflow as tf

from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
print("Tensorflow version " + tf.__version__)
import os
import pathlib
import PIL
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Conv2D, DepthwiseConv2D, ReLU, BatchNormalization, add,Softmax, AveragePooling2D, Dense, Input, GlobalAveragePooling2D, Activation, Reshape
from tensorflow.keras.models import Model
from keras import backend as K

Tensorflow version 2.7.0


In [ ]:
from zipfile import ZipFile

train_zip = 'drive/MyDrive/Datasets/traindata.zip'
val_zip = 'drive/MyDrive/Datasets/val_data.zip'
test_zip = 'drive/MyDrive/Datasets/test_data.zip'

with ZipFile(train_zip, 'r') as zip:
  zip.extractall("train_photos")

with ZipFile(val_zip, 'r') as zip:
  zip.extractall("val_photos")

with ZipFile(test_zip, 'r') as zip:
  zip.extractall("test_photos")
  
train_dir = os.path.join("./", 'train_photos')
val_dir = os.path.join("./", 'val_photos')
test_dir = os.path.join("./", 'test_photos')

# data_dir = pathlib.Path(train_dir)|
# PIL.Image.open(str(free[2]))

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 64 #64 cuz apparently needs to be 8? 

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    # width_shift_range=0.2,
    # #make moe brightness generation!
    # brightness_range=[0.2,1.0]
    # height_shift_range=0.2,
    # zoom_range=0.2,
    # horizontal_flip=True,
    # fill_mode="nearest",
)


val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=(BATCH_SIZE),
    # class_mode='binary'
    class_mode='categorical',
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=(BATCH_SIZE),
    # class_mode='binary'
    class_mode='categorical',
) 

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=(BATCH_SIZE),
    # subset='validation',
    # class_mode='binary'
    class_mode='categorical'
) 

Found 94493 images belonging to 2 classes.
Found 18647 images belonging to 2 classes.
Found 31825 images belonging to 2 classes.


In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.SGD(learning_rate=0.005, momentum=0.9), #original 0.0005
  loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
  metrics=['accuracy'])

In [ ]:
#V3 recommended training stats
model.compile(
  optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001, momentum=0.9),#,epsilon=0.002, rho=0.9), 
  loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
  metrics=['accuracy'])

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5, restore_best_weights=True)

rlrop = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience= 2, factor= 0.1, min_lr= 1e-4, verbose=1)

In [ ]:
hist = model.fit(
    test_generator,
    epochs=3, 
    # steps_per_epoch=steps_per_epoch,
    validation_data=val_generator,
    callbacks=[early_stop, rlrop],)

Epoch 1/3
285/498 [================>.............] - ETA: 44:46 - loss: 0.2846 - accuracy: 0.9566

In [ ]:
model.save("mobnettpu")

INFO:tensorflow:Assets written to: mobnettpu/assets


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [ ]:
!zip -r mobnettpu.zip mobnettpu


  adding: mobnettpu/ (stored 0%)
  adding: mobnettpu/saved_model.pb (deflated 92%)
  adding: mobnettpu/keras_metadata.pb (deflated 95%)
  adding: mobnettpu/assets/ (stored 0%)
  adding: mobnettpu/variables/ (stored 0%)
  adding: mobnettpu/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: mobnettpu/variables/variables.index (deflated 81%)


In [ ]:
def representative_data_gen():
  dataset_list = tf.data.Dataset.list_files(test_dir + '/*/*')
  for i in range(50):
    image = next(iter(dataset_list))
    image = tf.io.read_file(image)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE])
    image = tf.cast(image / 255., tf.float32)
    image = tf.expand_dims(image, 0)
    yield [image]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
# This enables quantization
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# This sets the representative dataset for quantization
converter.representative_dataset = representative_data_gen
# This ensures that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# For full integer quantization, though supported types defaults to int8 only, we explicitly declare it for clarity.
converter.target_spec.supported_types = [tf.int8]
# These set the input and output tensors to uint8 (added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
tflite_model = converter.convert()

with open('mobnettputest.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpjsbqe7sq/assets


INFO:tensorflow:Assets written to: /tmp/tmpjsbqe7sq/assets


In [ ]:
! curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -

! echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list

! sudo apt-get update

! sudo apt-get install edgetpu-compiler	

In [ ]:
! edgetpu_compiler mobnettputest.tflite


Edge TPU Compiler version 16.0.384591198
Started a compilation timeout timer of 180 seconds.

Model compiled successfully in 1853 ms.

Input model: mobnettputest.tflite
Input size: 3.82MiB
Output model: mobnettputest_edgetpu.tflite
Output size: 4.12MiB
On-chip memory used for caching model parameters: 3.86MiB
On-chip memory remaining for caching model parameters: 3.83MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 139
Operation log: mobnettputest_edgetpu.log
See the operation log file for individual operation details.
Compilation child process completed within timeout period.
Compilation succeeded! 


In [ ]:
def hard_swish(x):
  x = x+3.0
  return x * (ReLU(max_value=6)(x)/6.0)

def return_activation(x,name):
    x = Activation(hard_swish, name=name)(x)
    return x

In [ ]:
def expansion_block(x,t,filters,block_id):
    prefix = 'block_{}_'.format(block_id)
    total_filters = t*filters
    x = Conv2D(total_filters,1,padding='same',use_bias=False, name = prefix +'expand')(x)
    x = BatchNormalization(name=prefix +'expand_bn')(x)
    x = ReLU(6,name = prefix +'expand_relu')(x)
    # x = return_activation(x, name = prefix +'expand_relu')
    return x

def depthwise_block(x,stride,block_id):
    prefix = 'block_{}_'.format(block_id)
    x = DepthwiseConv2D(3,strides=(stride,stride),padding ='same', use_bias = False, name = prefix + 'depthwise_conv')(x)
    x = BatchNormalization(name=prefix +'dw_bn')(x)
    x = ReLU(6,name=prefix +'dw_relu')(x)
    # x = return_activation(x, name = prefix +'dw_relu')
    return x

def expansion_block_HS(x,t,filters,block_id):
    prefix = 'block_{}_'.format(block_id)
    total_filters = t*filters
    x = Conv2D(total_filters,1,padding='same',use_bias=False, name = prefix +'expand')(x)
    x = BatchNormalization(name=prefix +'expand_bn')(x)
    # x = ReLU(6,name = prefix +'expand_relu')(x)
    x = return_activation(x, name = prefix +'expand_relu')
    return x

def depthwise_block_HS(x,stride,block_id):
    prefix = 'block_{}_'.format(block_id)
    x = DepthwiseConv2D(3,strides=(stride,stride),padding ='same', use_bias = False, name = prefix + 'depthwise_conv')(x)
    x = BatchNormalization(name=prefix +'dw_bn')(x)
    # x = ReLU(6,name=prefix +'dw_relu')(x)
    x = return_activation(x, name = prefix +'dw_relu')
    return x

def projection_block(x,out_channels,block_id):
    prefix = 'block_{}_'.format(block_id)
    x = Conv2D(filters = out_channels,kernel_size = 1,padding='same',use_bias=False,name= prefix + 'compress')(x)
    x = BatchNormalization(name=prefix +'compress_bn')(x)
    return x

In [ ]:
def Bottleneck(x,t,filters, out_channels,stride,block_id):
    y = expansion_block(x,t,filters,block_id)
    y = depthwise_block(y,stride,block_id)
    y = projection_block(y, out_channels,block_id)
    if y.shape[-1]==x.shape[-1]:
        y = add([x,y])
    return y

def Bottleneck_HS(x,t,filters, out_channels,stride,block_id):
    y = expansion_block_HS(x,t,filters,block_id)
    y = depthwise_block_HS(y,stride,block_id)
    y = projection_block(y, out_channels,block_id)
    if y.shape[-1]==x.shape[-1]:
        y = add([x,y])
    return y


In [ ]:
def MobileNetV2(input_image = (224,224,3), n_classes=2):
    input = Input(input_image)

    x = Conv2D(32,kernel_size=3,strides=(2,2),padding = 'same', use_bias=False)(input)
    x = BatchNormalization(name='conv1_bn')(x)
    # x = ReLU(6, name = 'conv1_relu')(x)
    x = return_activation(x, name='initial_hs')


    # 17 Bottlenecks

    x = depthwise_block(x,stride=1,block_id=1)
    x = projection_block(x, out_channels=16,block_id=1)

    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 24, stride = 2,block_id = 2)
    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 24, stride = 1,block_id = 3)

    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 32, stride = 2,block_id = 4)
    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 32, stride = 1,block_id = 5)
    x = Bottleneck(x, t = 6, filters = x.shape[-1], out_channels = 32, stride = 1,block_id = 6)

    x = Bottleneck_HS(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 2,block_id = 7)
    x = Bottleneck_HS(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 1,block_id = 8)
    x = Bottleneck_HS(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 1,block_id = 9)
    x = Bottleneck_HS(x, t = 6, filters = x.shape[-1], out_channels = 64, stride = 1,block_id = 10)

    x = Bottleneck_HS(x, t = 6, filters = x.shape[-1], out_channels = 96, stride = 1,block_id = 11)
    x = Bottleneck_HS(x, t = 6, filters = x.shape[-1], out_channels = 96, stride = 1,block_id = 12)
    x = Bottleneck_HS(x, t = 6, filters = x.shape[-1], out_channels = 96, stride = 1,block_id = 13)

    x = Bottleneck_HS(x, t = 6, filters = x.shape[-1], out_channels = 160, stride = 2,block_id = 14)
    x = Bottleneck_HS(x, t = 6, filters = x.shape[-1], out_channels = 160, stride = 1,block_id = 15)
    x = Bottleneck_HS(x, t = 6, filters = x.shape[-1], out_channels = 160, stride = 1,block_id = 16)

    x = Bottleneck_HS(x, t = 6, filters = x.shape[-1], out_channels = 320, stride = 1,block_id = 17)

    #1*1 conv
    x = Conv2D(filters = 1280,kernel_size = 1,padding='same',use_bias=False, name = 'last_conv')(x)
    x = BatchNormalization(name='last_bn')(x)
    x = ReLU(6,name='last_relu')(x)

    #AvgPool 7*7
    x = GlobalAveragePooling2D(name='global_average_pool')(x)

    output = Dense(n_classes,activation='softmax')(x)

    model = Model(input, output)

    return model


In [ ]:
n_classes = 2
input_shape = (224,224,3)

model = MobileNetV2(input_shape,n_classes)
# model.summary()